# Data Processing

## Data Aggregation

In [2]:
import pandas as pd

In [25]:
# read the two csv files
tweet_df = pd.read_csv("../data/tweet_data.csv")
stock_df = pd.read_csv("../data/stock_data.csv")

In [26]:
# transform the date column in the stock data to datetime and keep the date 
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df['Date'] = stock_df['Date'].dt.date

In [27]:
# Transform the date column in the tweet data to datetime and keep the date
tweet_df['Date'] = pd.to_datetime(tweet_df['Date'])
tweet_df['Date'] = tweet_df['Date'].dt.date

In [28]:
# drop the full company name column from the tweet data
tweet_df.drop('Company Name', axis=1, inplace=True)

In [29]:
# print the columns of the two dataframes
print("Tweet Data Columns: ")
for col in tweet_df.columns:
    print(col)
print()
print("Stock Data Columns: ")
for col in stock_df.columns:
    print(col)

Tweet Data Columns: 
Date
Tweet
Stock Name

Stock Data Columns: 
Date
Open
High
Low
Close
Adj Close
Volume
Stock Name


In [30]:
# check how many dates figure in both dataframes
print("Number of dates in both data: ", len(set(tweet_df['Date'].unique()) & set(stock_df['Date'].unique())))

Number of dates in both data:  252


In [36]:
# join the two dataframes on the date column
joined_df = pd.merge(tweet_df, stock_df, 
                     on=['Date', 'Stock Name'], 
                     how='left')

In [40]:
# print the columns of the joined dataframe
print("Joined Data Columns: ")
for col in joined_df.columns:
    print(col)

# print the shape of the joined dataframe
print("Joined Data Shape: ", joined_df.shape)

Joined Data Columns: 
Date
Tweet
Stock Name
Open
High
Low
Close
Adj Close
Volume
Joined Data Shape:  (80793, 9)


In [41]:
# save the new dataframe to a csv file
joined_df.to_csv("../data/joined_data.csv", index=False)

## Data Cleaning

In [3]:
# read the new dataframe from the csv file
df = pd.read_csv("../data/joined_data.csv")

### Missing Values

In [4]:
# get a rundown of the missing values per column
print("Missing Values: ")
for col in df.columns:
    print(col, df[col].isnull().sum())

Missing Values: 
Date 0
Tweet 0
Stock Name 0
Open 17117
High 17117
Low 17117
Close 17117
Adj Close 17117
Volume 17117


In [5]:
# drop the rows with missing values
df.dropna(inplace=True)

In [6]:
# print the shape of the dataframe
print("New shape of the dataframe: ", df.shape)

New shape of the dataframe:  (63676, 9)


### Tweet Cleaning

In [7]:
import emoji
import re

In [8]:
# get a list of the stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\elaty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
def split_hashtag(hashtag):
    # Splitting based on the presence of capital letters (camel case)
    parts = re.findall('[A-Z][^A-Z]*', hashtag)
    # If the hashtag is all lowercase, just return it as is.
    if not parts:
        return hashtag
    # Re-join the parts with a space
    return ' '.join(part for part in parts)

# Process hashtags
def replace_with_processed(match):
    hashtag = match.group(1)
    return split_hashtag(hashtag)

# create a function to clean the text of a tweet
def clean_tweet(tweet):
    # Convert emojis to words
    tweet = emoji.demojize(tweet)

    # Replace hashtags in the tweet with their processed forms
    tweet = re.sub(r'#(\w+)', replace_with_processed, tweet)
    
    # Lowercase the tweet
    tweet = tweet.lower()

    # Remove stopwords (optional, based on your preference)
    tweet = ' '.join([word for word in tweet.split() if word not in stop_words])
    
    # Remove URLs
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet, flags=re.MULTILINE)
    
    # Remove user @ references
    tweet = re.sub(r'\@\w+','', tweet)
    
    # Remove punctuations and special characters (except $ and %)
    tweet = re.sub(r'[^\w\s\$%]', '', tweet)
    
    return tweet

In [10]:
# run the clean_tweet function on the tweet text column
df['Tweet'] = df['Tweet'].apply(clean_tweet)

In [11]:
print("Cleaned Tweet Example: ", df['Tweet'][0])

Cleaned Tweet Example:  mainstream media done amazing job brainwashing people today work asked companies believe amp said  make safest cars amp everyone disagreed heardthey catch fire amp batteries cost 20k replace


In [12]:
# save the cleaned dataframe to a csv file
df.to_csv("../data/cleaned_data.csv", index=False)

## Data Preprocessing

In [25]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import joblib

In [20]:
# read the cleaned data from the csv file
df = pd.read_csv("../data/cleaned_data.csv")

In [21]:
# create our scaler objects
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

In [22]:
# for each of the columns in [Open, High, Low, Close, Adj Close, Volume], apply the scalers
for col in ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']:
    df[col] = minmax_scaler.fit_transform(df[col].values.reshape(-1, 1))
    df[col] = standard_scaler.fit_transform(df[col].values.reshape(-1, 1))

In [23]:
# for the Volume column, apply the log transformation
df['Volume'] = np.log(df['Volume'] + 1)

c:\Users\elaty\AppData\Local\Programs\Python\Venvs\Machine Learning\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [24]:
# print the first 5 rows of the dataframe
print(df.head())

         Date                                              Tweet Stock Name  \
0  2022-09-29  mainstream media done amazing job brainwashing...       TSLA   
1  2022-09-29  tesla delivery estimates around 364k analysts ...       TSLA   
2  2022-09-29  3 even include 630m unvested rsus 630 addition...       TSLA   
3  2022-09-29     hahaha still trying stop tesla fsd bro get ...       TSLA   
4  2022-09-29         stop trying kill kids sad deranged old man       TSLA   

       Open      High       Low    Close  Adj Close   Volume  
0  0.506929  0.454661  0.411441  0.37235    0.37616  0.25447  
1  0.506929  0.454661  0.411441  0.37235    0.37616  0.25447  
2  0.506929  0.454661  0.411441  0.37235    0.37616  0.25447  
3  0.506929  0.454661  0.411441  0.37235    0.37616  0.25447  
4  0.506929  0.454661  0.411441  0.37235    0.37616  0.25447  


In [26]:
# save the scaler objects to disk
joblib.dump(minmax_scaler, '../models/minmax_scaler.pkl')
joblib.dump(standard_scaler, '../models/standard_scaler.pkl')

['../models/standard_scaler.pkl']

In [ ]:
# save the dataframe to a csv file
df.to_csv("../data/scaled_data.csv", index=False)